In [15]:
import pandas as pd
import os
from deeper.data import data2Inputs
from sklearn.metrics import f1_score
import numpy as np

In [4]:
DATASET_DIR = 'itunes_amazon'
train_df = pd.read_csv(os.path.join(DATASET_DIR,'train.csv'))
validation_df = pd.read_csv(os.path.join(DATASET_DIR,'valid.csv'))
test_df = pd.read_csv(os.path.join(DATASET_DIR,'test.csv'))

In [5]:
all_df = pd.concat([train_df,validation_df,test_df])
all_df.head()

,ltable_id,rtable_id,label
0,1409,51560,0
1,1289,34035,0
2,2666,35475,1
3,1157,27989,0
4,3332,7412,0


In [7]:
positives_df = all_df[all_df.label==1]
positives_df = positives_df.drop(['label'], axis=1)
positives_df.columns = ['id1','id2']

In [9]:
positives_df.to_csv('itunes_amazon/matches_itunes_amazon.csv',index=False)

## Wrapper function for Deeper

In [18]:
def dataset_tolist(dataset,lprefix,rprefix):
    res = []
    attributes = [col for col in dataset.columns if col not in ['label','id']]
    for i in range(len(dataset.index)):
        row = dataset.iloc[i]
        ltuple,rtuple = ([],[])
        for attr in attributes:
            if attr.startswith(lprefix):
                ltuple = ltuple+ str(row[attr]).split()
            else:
                rtuple = rtuple+str(row[attr]).split()
        res.append((ltuple,rtuple,row['label']))
    return res

In [5]:
def wrap_deeper(dataset,lprefix,rprefix,model,tokenizer,embedding_model):
    dataset_list = dataset_tolist(dataset,lprefix,rprefix)
    ltokens,rtokens,_ = data2Inputs(dataset_list,tokenizer)
    lembeddings,rembeddings = embedding_model.predict([ltokens,rtokens])
    predictions = model.predict([lembeddings,rembeddings])
    return predictions

## Test

In [6]:
from keras.models import load_model
from deeper.DeepER import init_embeddings_index, init_embeddings_model

In [7]:
EMBEDDING_FILEPATH ='embeddings/glove.840B.300d.txt'
# Caricamento strutture dati e modelli ausiliari.
embeddings_index = init_embeddings_index(EMBEDDING_FILEPATH)
emb_dim = len(embeddings_index['cat']) # :3
embeddings_model, tokenizer = init_embeddings_model(embeddings_index)

* Costruzione indice degli embeddings.....

2196017it [02:21, 15540.13it/s]

Fatto. 2196016 embeddings totali.
* Creazione del modello per il calcolo degli embeddings....
* Inizializzo il tokenizzatore.....

Fatto: 1702926 parole totali.
* Preparazione della matrice di embedding.....Fatto. Dimensioni matrice embeddings: (1702927, 300)

°°° EMBEDDING MODEL °°°
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Tupla_A (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
Tupla_B (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
Embedding_lookup (Embedding)    (None, None, 300)    510878100   Tupla_A[0][0]                    
                                                                 Tupla_B[0][0]                    
Total params: 510,878,100
Trainable p

In [8]:
walmart_model = load_model('models/DeepER_best_model_100_walmart-amazon.h5')

In [19]:
test_df = pd.read_csv('test-walmart-amazon.csv')
predictions = wrap_deeper(test_df,'ltable_','rtable_',walmart_model,tokenizer,embeddings_model)

* Preparazione input......Fatto. 2049 tuple totali, esempio label: 0 -> [1. 0.], Table1 shape: (2049, 34), Table2 shape: (2049, 32)


In [20]:
predicted_labels = np.argmax(predictions,axis=1)
true_labels = list(map(lambda s:int(s),test_df.label.values))
f1_score(true_labels,predicted_labels)

0.18547595682041215

In [12]:
itunes_model = load_model('models/DeepER_best_model_100_itunes-amazon.h5')
test_df = pd.read_csv('augmented_test.csv',dtype=str).fillna("")
predictions = wrap_deeper(test_df,'ltable_','rtable_',itunes_model,tokenizer,embeddings_model)

* Preparazione input......Fatto. 180 tuple totali, esempio label: 0 -> [1. 0.], Table1 shape: (180, 67), Table2 shape: (180, 66)


In [13]:
predicted_labels = np.argmax(predictions,axis=1)
true_labels = list(map(lambda s:int(s),test_df.label.values))
f1_score(true_labels,predicted_labels)

0.5952380952380952